In [ ]:
%run init.ipynb

In [2]:
import numpy as np
from json import JSONEncoder

class NpEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.66.200:37809/status,
Dashboard: http://192.168.66.200:37809/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.66.200:40809,Workers: 0
Dashboard: http://192.168.66.200:37809/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

In [5]:
import dask
import dask.dataframe as dd

indexed_dir = f"{log_dir}/indexed/file_id"
indexed_ddf = dd.read_parquet(f"{indexed_dir}/*.parquet", calculate_divisions=True, index=['file_id'])
indexed_ddf

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,proc_id
npartitions=2286,,,,,,,,,,,,,,,,,,
-9223371986035283781,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,float32,float32,int64,int64
-8872388567018363149,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8752386220556340459,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9223369538921024184,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
indexed_ddf.index.count().compute()

In [ ]:
%%time

import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
from dask.diagnostics import ProgressBar
from time import sleep

def clear_logs(dask_scheduler): # As suggested in #3898 
    dask_scheduler.log.clear()
    dask_scheduler.transition_log.clear()
    dask_scheduler.events.clear()

def read_global_json(log_dir):
    with open(f"{log_dir}/filenames.json") as file:
        procs = json.load(file)
    return np.array(procs)

def proc_metrics(index, filenames, ddf):
#     res = []
    print("filenames", len(filenames))
    dask.distributed.get_worker().log_event("filenames", len(filenames))
    dask.distributed.get_worker().log_event("filenames_shape", np.array(filenames).shape)
#     dask.distributed.get_worker().log_event("procs3_type", type(procs))
    filename = f"{log_dir}/metrics/file_id/{index}.parquet"
    splice_ddf = ddf.loc[filenames].reset_index()
    target_ddf = splice_ddf.compute()
    aggregate = target_ddf.groupby(['index','io_cat']).agg({'duration':sum, 
                                                              'size':sum, 
                                                              'index':'count',
                                                              'filename':min})
    aggregate.reset_index(inplace=True)
    aggregate.columns  = ['_'.join(col) for col in aggregate.columns.values]
    aggregate.to_parquet(filename)
    return filename
#     for f in filenames:
#         print(p)
#         print(p.compute())
#         dask.distributed.get_worker().log_event("p", {"p": p})
#         target_ddf = ddf.loc[f:f].reset_index().compute()
#         filter_d = proc_metrics_filter(p, proc_ddf_d)
#         res.append({'f':int(f),'n_f':len(target_ddf.index)})
#     return res

client.run_on_scheduler(clear_logs)

n_el = 21260259
        
tasks = []
# cm1_ddf_d = dask.delayed(lambda log_dir: dd.read_parquet(f"{log_dir}/*.parquet"))(log_dir)
# cm1_indexed_d = dask.delayed(lambda ddf: ddf.set_index(['proc_id']))(cm1_ddf_d)
cm1_indexed_d = dask.delayed(lambda indexed_dir: dd.read_parquet(f"{indexed_dir}/*.parquet", calculate_divisions=True, index=['file_id']))(indexed_dir)

cm1_persisted_d = dask.delayed(lambda ddf: ddf.persist())(cm1_indexed_d)
cm1_files_d = dask.delayed(lambda log_dir: read_global_json(log_dir))(log_dir)
cm1_files_a = da.from_delayed(cm1_files_d, shape=(n_el,), dtype=int)
cm1_files_a = cm1_files_a.rechunk(math.ceil(math.sqrt(n_el)))

delayed_result = [
    dask.delayed(proc_metrics)(index, filenames, cm1_persisted_d) 
    for index, filenames in enumerate(cm1_files_a.blocks.ravel())
]

result = dask.delayed(list)(delayed_result)
# result.visualize("mapped.png")
    
metrics = result.compute()

print(len(metrics))
print(metrics)

# with open(f"gen_file_metrics.json", "w+") as file:
#     json.dump(metrics, file, cls=NpEncoder)

In [ ]:
client.get_events("filenames")